# Monitor

In this step we will create Monitor workflow pipeline based on schedule, whose purpose is to call

1) call the service created after training a data prediction model using darts framework and NBEATS Deep Learning model. (see notebook parcheggi_ml.ipynb)
2) save the prediction in database.


## Platform Support - Data Ops
We use the platform support to read the data created into the platform after the execution of notebook(parcheggi_data_pipeline.ipynb) for training

In [5]:
import pandas as pd
import requests
import os
import json
import digitalhub as dh
import getpass as gt

In [6]:
PROJECT_NAME = "parcheggi-nk-scheduler-"+gt.getuser()
proj = dh.get_or_create_project(PROJECT_NAME)
print("created project {}".format(PROJECT_NAME))
PROJECT_NAME

created project parcheggi-nk-scheduler-digitalhubdev


'parcheggi-nk-scheduler-digitalhubdev'

In [3]:
serving_fn = proj.get_function('serve_model')

In [4]:
data_item_download = proj.get_dataitem("dataset")

In [5]:
parkings_df = data_item_download.as_df()

In [6]:
parkings_df

,parcheggio,data,posti_liberi,posti_occupati,posti_totali,lat,lon
0,VIII Agosto,2024-06-07T01:59:00+00:00,484.0,141.0,625,44.500297,11.345368
1,Riva Reno,2024-06-07T02:09:00+00:00,369.0,101.0,470,44.501153,11.336062
2,Riva Reno,2024-06-07T02:19:00+00:00,369.0,101.0,470,44.501153,11.336062
3,VIII Agosto,2024-06-07T02:29:00+00:00,487.0,138.0,625,44.500297,11.345368
4,Riva Reno,2024-06-07T02:29:00+00:00,369.0,101.0,470,44.501153,11.336062
...,...,...,...,...,...,...,...
46202,VIII Agosto,2024-09-30T13:19:00+00:00,150.0,475.0,625,44.500297,11.345368
46203,Riva Reno,2024-09-30T13:19:00+00:00,262.0,208.0,470,44.501153,11.336062
46204,Autostazione,2024-09-30T13:19:00+00:00,150.0,115.0,265,44.504422,11.346514
46205,Autostazione,2024-09-30T13:39:00+00:00,142.0,123.0,265,44.504422,11.346514


In [7]:
pd. __version__

'1.4.4'

In [8]:
parkings_df = data_item_download.as_df()
parkings_df

,parcheggio,data,posti_liberi,posti_occupati,posti_totali,lat,lon
0,VIII Agosto,2024-06-07T01:59:00+00:00,484.0,141.0,625,44.500297,11.345368
1,Riva Reno,2024-06-07T02:09:00+00:00,369.0,101.0,470,44.501153,11.336062
2,Riva Reno,2024-06-07T02:19:00+00:00,369.0,101.0,470,44.501153,11.336062
3,VIII Agosto,2024-06-07T02:29:00+00:00,487.0,138.0,625,44.500297,11.345368
4,Riva Reno,2024-06-07T02:29:00+00:00,369.0,101.0,470,44.501153,11.336062
...,...,...,...,...,...,...,...
46202,VIII Agosto,2024-09-30T13:19:00+00:00,150.0,475.0,625,44.500297,11.345368
46203,Riva Reno,2024-09-30T13:19:00+00:00,262.0,208.0,470,44.501153,11.336062
46204,Autostazione,2024-09-30T13:19:00+00:00,150.0,115.0,265,44.504422,11.346514
46205,Autostazione,2024-09-30T13:39:00+00:00,142.0,123.0,265,44.504422,11.346514


In [1]:
# import datetime 
# import requests
# import json
# import pandas as pd


# # Define a custom function to serialize datetime objects 
# def serialize_datetime(obj): 
#     if isinstance(obj, datetime.datetime): 
#         return obj.isoformat() 
#     raise TypeError("Type not serializable") 

# # get current date and time as string
# date_str = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
# parkings_df = data_item_download.as_df()
# # initialize an empty dataframe for predictions
# pred_df = pd.DataFrame(columns=['parcheggio', 'datetime', 'predicted_mean'])
# # iterate over each parking in the dataset
# for index, row in parkings_df.iterrows():
#     parking_str = row['parcheggio']
#     # construct API URL based on parking and current date
#     API_URL = f'https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/disponibilita-parcheggi-storico/records?where=data%3C%3D%27{date_str}%27%20and%20parcheggio%3D%27{parking_str}%27&order_by=data%20DESC&limit=100'
#     # define the file to store the latest data
#     latest_data_file = 'last_records.json'
#     # fetch data from the API and save it to a file
#     with requests.get(API_URL) as r:
#         with open(latest_data_file, "wb") as f:
#             f.write(r.content)
#         # read the latest data from the file and process it
#         with open(latest_data_file) as f:
#             json_data = json.load(f)
#             df_latest = pd.json_normalize(json_data['results']).drop(columns=['guid', 'occupazione']).rename(columns={"coordinate.lon": "lon", "coordinate.lat": "lat"})
#             df_latest.data = df_latest.data.astype('datetime64')
#             df_latest['value'] = df_latest.posti_occupati / df_latest.posti_totali
#             df_latest['date'] = df_latest.data.dt.round('30min')
#             df_latest = df_latest.drop(columns=['parcheggio'])
#             df_latest = df_latest.groupby('date').agg({'value': 'mean'})

#         # convert the processed data to JSON and make a request to the serving predictor function
#         jsonstr = df_latest.reset_index().to_json(orient='records')
#         arr = json.loads(jsonstr)
#         print(arr)
      

In [20]:
%%writefile "src/monitor_parking_predictions.py"
from digitalhub_runtime_python import handler
import datetime 
import requests
import json
import pandas as pd

@handler(outputs=["parking_data_predicted_model"])
def monitor_predictor(project,  parkings_di):
    """
    Monitor and predict parking occupancy.
    """

    # get serving predictor function
    run_serve_model = project.get_function('serve_model')
    run_serve_model.refresh

    # get current date and time as string
    date_str = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

    # get parkings dataset and convert it to a dataframe
    parkings_df = parkings_di.as_df()

    # initialize an empty dataframe for predictions
    pred_df = pd.DataFrame(columns=['parcheggio', 'datetime', 'predicted_mean'])

    # iterate over each parking in the dataset
    for index, row in parkings_df.iterrows():
        parking_str = row['parcheggio']

        # construct API URL based on parking and current date
        API_URL = f'https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/disponibilita-parcheggi-storico/records?where=data%3C%3D%27{date_str}%27%20and%20parcheggio%3D%27{parking_str}%27&order_by=data%20DESC&limit=100'

        # define the file to store the latest data
        latest_data_file = 'last_records.json'

        # fetch data from the API and save it to a file
        with requests.get(API_URL) as r:
            with open(latest_data_file, "wb") as f:
                f.write(r.content)

        # read the latest data from the file and process it
        with open(latest_data_file) as f:
            json_data = json.load(f)
            df_latest = pd.json_normalize(json_data['results']).drop(columns=['guid', 'occupazione']).rename(columns={"coordinate.lon": "lon", "coordinate.lat": "lat"})
            df_latest.data = df_latest.data.astype('datetime64')
            df_latest['value'] = df_latest.posti_occupati / df_latest.posti_totali
            df_latest['date'] = df_latest.data.dt.round('30min')
            df_latest = df_latest.drop(columns=['parcheggio'])
            df_latest = df_latest.groupby('date').agg({'value': 'mean'})

        # convert the processed data to JSON and make a request to the serving predictor function
        jsonstr = df_latest.reset_index().to_json(orient='records')
        arr = json.loads(jsonstr)
        SERVICE_URL = run_serve_model.status.to_dict()["service"]["url"]
        with requests.post(f'http://{SERVICE_URL}', json={"inference_input":arr}) as r:
            res = r.content
        res_df = pd.DataFrame(res['outputs'])
        res_df['datetime'] = res_df['date'].astype('datetime64')
        res_df['parcheggio'] = parking_str
        res_df['predicted_mean'] = res_df['value']
        res_df = res_df.drop(columns=['date', 'value'])
        pred_df = pd.concat([pred_df, res_df], ignore_index=True)

    # concatenate the predicted results with the existing data (if any) and remove duplicates
    old_pd = pred_df
    try: 
        dat_old = project.get_dataitem('store://datasets/parcheggi/parking_prediction_nbeats_model')
        old_pd = pd.concat([dat_old.as_df(), pred_df], ignore_index=True)
        old_pd = old_pd.drop_duplicates(subset=['parcheggio', 'datetime'])
    except: pass

    # log the predictions as a dataset in the project
    project.log_dataitem('parking_prediction_nbeats_model', old_pd, stats=True)

    old_pd = pred_df.copy()
    old_pd['slice_datetime'] = date_str
    try:
        dat_old = project.get_dataitem('store://datasets/parcheggi/parking_prediction_nbeats_model_sliced')
        old_pd = pd.concat([dat_old.as_df(), old_pd], ignore_index=True)
    except: pass

    # log the predictions as a dataset in the project
    project.log_dataitem('parking_prediction_nbeats_model_sliced', old_pd, stats=True)

Writing src/monitor_parking_predictions.py


In [22]:
func = proj.new_function(name="monitor-parking-prediction",
                         kind="python",
                         python_version="PYTHON3_9",
                         source={"source": "src/monitor_parking_predictions.py", "handler": "monitor_predictor"})

In [24]:
data_item_download = proj.get_dataitem("dataset").key
run_monitor_parkings = func.run(action="job",inputs={"parkings_di": data_item_download},outputs={})